# <font color='black'> Регрессионный анализ панельных данных и каузальность, 2026 </font>
## <font color='black'> Анализ панельных данных. Модели с фиксированными эффектами (FE-models) и модели со случайными эффектами </font>
Панельные данные имеют как пространственное, так и временное измерение. К примеру, наблюдения для одной страны представлены за несколько лет. При такой структуре данных наивно было бы полагать, что условие о независимости наблюдений соблюдается. Наблюдения "внутри" одной пространственной единицы, разумеется, связаны. В этом случае мы не можем применять классическую линейную регрессию. В качестве альтернативы выступают широко применяемые модели с фиксированными эффектами.

Кратко о данных, с которыми мы сегодня будем работать.

*Источник: Массив данных основан на идее исследования Ehrlich I., Lui F. (1999) Bureaucratic Corruption and Endogenous Economic Growth. The Journal of Political Economy, 107 (6), pp. 270 – 293.*

* country -  Страна
* countrygroup - Страны в датасете разделены на три группы. Первая группа (countrygroup = 1) включает Австралию, Канаду, Японию, Новую Зеландию и США. Вторая группа (countrygroup = 2) включает страны Восточной Европы (Болгария, Хорватия, Чешская Республика, Эстония, Венгрия, Латвия, Польша, Словения, Словакия). В третью группу в основном входят страны Западной Европы
* year - Год. Данные охватывают временной промежуток с 2006 по 2015 год
* pol_stab - Политическая стабильность и отсутствие насилия/терроризма. Отражает склонность к политической нестабильности и/или политически мотивированному насилию, включая терроризм. Значения показателя варьируются от −2.5 до 2.5, причем более высокие значения означают более высокий уровень политической стабильности. Источником данных являются Индикаторы качества государственного управления (WGI). В рамках анализа будем использовать этот показатель в качестве зависимой переменной
* con_cor - Показатель контроля коррупции. Отражает восприятие степени использования государственной власти для личной выгоды, включая как мелкие, так и крупные формы коррупции. Показатель изменяется от −2.5 до 2.5, причем более высокие зна-чения соответствуют более низкому уровню коррупции (т.е. более высокому контролю). Источником данных является WGI. В рамках анализа будем использовать эту переменную в качестве ключевого предиктора
* herfgov_DPI - Индекс концентрации Херфиндаля. В контексте данного исследования он используется для измерения уровня конкуренции между политическими партиями. Шкала варьируется от 0 до 1, где 1 означает отсутствие конкуренции. В рамках анализа будем использовать как контрольную переменную
* govt_consump_WDI - Государственные расходы на конечное потребление (в процентах от ВВП). В рамках анализа будем использовать как контрольную переменную


Подгрузим необходимые библиотеки и откроем массив "RAPDC_lab1.dta".

In [50]:
import pandas as pd
import statsmodels.formula.api as statf
import numpy as np
import numpy.linalg as la
!pip install linearmodels
from linearmodels import PanelOLS
from linearmodels import RandomEffects
from scipy import stats

In [51]:
dta = pd.read_stata("RAPDC_lab1.dta")
dta = dta.dropna()
dta.head()

,year,country,pol_stab,con_cor,herfgov_DPI,govt_consump_WDI,countrygroup
0,2006,Australia,0.935188,1.960568,0.75987,17.288634,1.0
1,2007,Australia,0.928790,2.010918,0.75987,17.175808,1.0
2,2008,Australia,0.955645,2.042482,1.00000,17.096951,1.0
3,2009,Australia,0.855689,2.051661,1.00000,17.496081,1.0
4,2010,Australia,0.888860,2.031455,1.00000,17.959095,1.0


Выведем описательные статистики для ключевых переменных, но по подгруппам стран.

In [52]:
dta.groupby('countrygroup')['pol_stab'].describe()

,count,mean,std,min,25%,50%,75%,max
countrygroup,,,,,,,,
1.0,50.0,0.976247,0.269992,0.375730,0.885950,1.002432,1.123308,1.525453
2.0,80.0,0.755896,0.272767,0.017322,0.608071,0.765232,0.981037,1.147953
3.0,209.0,0.830497,0.470127,-0.473777,0.508217,0.940750,1.234161,1.512313


In [53]:
dta.groupby('countrygroup')['con_cor'].describe()

,count,mean,std,min,25%,50%,75%,max
countrygroup,,,,,,,,
1.0,50.0,1.818398,0.365734,1.238238,1.420593,1.886696,2.049905,2.391192
2.0,80.0,0.415402,0.418033,-0.267288,0.142534,0.357439,0.683832,1.303456
3.0,209.0,1.413227,0.780179,-0.262550,0.960238,1.575072,2.118235,2.469991


В нашей первой модели мы учтем разные "стартовые" условия. Для этого добавим дамми-переменные на страну. Возьмем в предварительном варианте стандартные ошибки, состоятельные в условиях гетероскедастичности. Они пока не учитывают автокорреляцию, тем не менее, позволяют отследить, есть ли различия в средних по зависимой переменной между странами (см. скорректированные стандартные ошибки для оценок коэффициентов при дамми-переменных)

Проинтерпретируйте оценки model1. Почему в данном случае мы получаем большой R-squared? Можно ли на него полагаться?

In [54]:
model1 = statf.ols(formula = 'pol_stab ~ con_cor + govt_consump_WDI + herfgov_DPI + C(country)', data = dta).fit(cov_type = "HC3")
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:               pol_stab   R-squared:                       0.905
Model:                            OLS   Adj. R-squared:                  0.894
Method:                 Least Squares   F-statistic:                     100.3
Date:                Mon, 19 Jan 2026   Prob (F-statistic):          2.10e-146
Time:                        22:10:09   Log-Likelihood:                 221.05
No. Observations:                 339   AIC:                            -368.1
Df Residuals:                     302   BIC:                            -226.5
Df Model:                          36                                         
Covariance Type:                  HC3                                         
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept       

Можем также использовать кластеризованные по странам стандартные ошибки для того, чтобы учесть возможную корреляцию между наблюдениями внутри одной пространственной единицы. Однако в этом случае мы не можем полагаться на стандартные ошибки, полученные для дамми-переменных. Поэтому, как правило, кластеризованные стандартные ошибки применяются в модели с внутригрупповым преобразованием (оценим данную модель далее)   

In [55]:
model1_clustered_se = model1.get_robustcov_results(cov_type='cluster', groups=dta['country'])
print(model1_clustered_se.summary())

                            OLS Regression Results                            
Dep. Variable:               pol_stab   R-squared:                       0.905
Model:                            OLS   Adj. R-squared:                  0.894
Method:                 Least Squares   F-statistic:                     7.002
Date:                Mon, 19 Jan 2026   Prob (F-statistic):           0.000901
Time:                        22:10:09   Log-Likelihood:                 221.05
No. Observations:                 339   AIC:                            -368.1
Df Residuals:                     302   BIC:                            -226.5
Df Model:                          36                                         
Covariance Type:              cluster                                         
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept       

/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 36, but rank is 3
  warnings.warn('covariance of constraints does not have full '


Как Вы уже могли заметить выше, Python по умолчанию в качестве базовой категории (относительно которой происходит сравнение) выбирает первую пространственную единицу. При этом не всегда удобно интерпретировать относительно первой категории. Ниже рассмотрим, как можно изменить базовую категорию.

* Что в выдаче изменилось? Изменилась ли оценка коэффициента при контроле коррупции?
* Проинтерпретируйте оценки model1_2

In [56]:
model1_2 = statf.ols(formula = 'pol_stab ~ con_cor + govt_consump_WDI + herfgov_DPI + C(country, Treatment("Denmark"))', data = dta).fit(cov_type = "HC3")
print(model1_2.summary())

                            OLS Regression Results                            
Dep. Variable:               pol_stab   R-squared:                       0.905
Model:                            OLS   Adj. R-squared:                  0.894
Method:                 Least Squares   F-statistic:                     100.3
Date:                Mon, 19 Jan 2026   Prob (F-statistic):          2.10e-146
Time:                        22:10:10   Log-Likelihood:                 221.05
No. Observations:                 339   AIC:                            -368.1
Df Residuals:                     302   BIC:                            -226.5
Df Model:                          36                                         
Covariance Type:                  HC3                                         
                                                         coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------

На следующем шаге оценим модель с внутригрупповым преобразованием. Вспомните, в чем ее отличие от LSDV-модели (модели с дамми-переменными), почему часто используется именно такая спецификация с внутригрупповым преобразованием. Обратите внимание на то, что для того, чтобы в Python оценить FE-модели и RE-модели, нужно предварительно указать, какая переменная показывает пространственное измерение, а какая - временное.
* Проинтерпретируйте оценки model2
* Проинтерпретируйте значения F-test for Poolability

In [57]:
dta_index = dta.set_index(['country', 'year'])

In [58]:
model2 = PanelOLS.from_formula('pol_stab ~ con_cor + govt_consump_WDI + herfgov_DPI + EntityEffects', data = dta_index).fit(cov_type='clustered', cluster_entity=True, group_debias=True)
print(model2)

                          PanelOLS Estimation Summary                           
Dep. Variable:               pol_stab   R-squared:                        0.0833
Estimator:                   PanelOLS   R-squared (Between):              0.1598
No. Observations:                 339   R-squared (Within):               0.0833
Date:                Mon, Jan 19 2026   R-squared (Overall):              0.1589
Time:                        22:10:10   Log-likelihood                    221.05
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      9.1488
Entities:                          34   P-value                           0.0000
Avg Obs:                       9.9706   Distribution:                   F(3,302)
Min Obs:                       9.0000                                           
Max Obs:                      10.0000   F-statistic (robust):             1.5347
                            

Сравним полученные оценки с результатами оценивания модели со случайными эффектами.
* Вспомните допущения данной модели и оцените критически
* Какой метод оценивания используется в данном случае?

In [59]:
model3 = RandomEffects.from_formula('pol_stab ~ con_cor + govt_consump_WDI + herfgov_DPI', data = dta_index).fit(cov_type='clustered', cluster_entity=True, group_debias=True)
print(model3)

                        RandomEffects Estimation Summary                        
Dep. Variable:               pol_stab   R-squared:                        0.4276
Estimator:              RandomEffects   R-squared (Between):              0.8826
No. Observations:                 339   R-squared (Within):               0.0293
Date:                Mon, Jan 19 2026   R-squared (Overall):              0.8654
Time:                        22:10:10   Log-likelihood                    194.29
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      83.655
Entities:                          34   P-value                           0.0000
Avg Obs:                       9.9706   Distribution:                   F(3,336)
Min Obs:                       9.0000                                           
Max Obs:                      10.0000   F-statistic (robust):             98.222
                            

Для начала приведите содержательные основания для выбора FE- или RE-модели: какой модели Вы отдадите предпочтение в данном случае?

Но мы также обсуждали с Вами тест Хаусмана и его ограничения для сравнения FE- и RE-модели.
Зададим в явном виде статистику для проверки гипотезы  и ее распределение: $$S = (\hat{b}_{fe} - \hat{b}_{re})^T(Cov({\hat{b}_{fe}}) - Cov(\hat{b}_{re}))^{-1}(\hat{b}_{fe} - \hat{b}_{re}) \sim \chi^2_{k}$$, где k - количество предикторов. Проинтерпретируйте результаты тестирования

In [60]:
X_varying = ['con_cor', 'govt_consump_WDI', 'herfgov_DPI']

fe_coef = model2.params
re_coef = model3.params.loc[X_varying]

fe_cov = model2.cov
re_cov = model3.cov[X_varying].loc[X_varying]

hausman = (fe_coef - re_coef).dot(la.linalg.pinv(fe_cov - re_cov).dot(fe_coef - re_coef))
df = fe_coef.size
pvalue = stats.chi2.sf(hausman, df)
print('Chi-squared:', hausman, '\n' 'P-value:', pvalue, '\n' 'Degrees of freedom:', df)

Chi-squared: 2.938684126670331 
P-value: 0.4011758121086503 
Degrees of freedom: 3


*Процедура взвешивания*

Получим теперь оценку коэффициента при показателе контроля коррупции в FE-модели, однако посредством деления массива на подвыборки. Страны и будут составлять наши подвыборки. На первом шаге посчитаем разброс значений контроля коррупции по подгруппам.

In [61]:
n = dta.groupby('country', sort = False).size()
varbygroups = dta.groupby('country', sort = False).var().con_cor*(n-1)

varbygroups.sort_values(ascending = False)

,0
country,
Spain,0.435830
Austria,0.418686
Slovakia,0.346133
Greece,0.292173
Iceland,0.252657
Hungary,0.250408
Japan,0.229801
Italy,0.227452
Poland,0.181022


На втором шаге оценим парную линейную регрессию политической стабильности на уровень коррупции на подвыборках разных стран и сохраним оценки коэффициентов при предикторе.

In [62]:
coefconcor = []

for i in dta.country.unique():
  subsample = dta[dta.country == i]
  coefconcor.append(statf.ols(formula = 'pol_stab ~ con_cor', data = subsample).fit().params.iloc[1])

Третий шаг - посчитаем сумму взвешенных оценок коэффициентов. Вес для i-ой страны - отношение внутригруппового разброса значений показателей контроля коррупции i-ой страны к сумму соответствующих внутригрупповых разбросов по всем странам.

И вуаля! Получили ту же оценку коэффициента посредством процедуры взвешивания. Чем нам с практической точки зрения полезны эти результаты?

In [63]:
weightedsum = sum(coefconcor*varbygroups/sum(varbygroups))
weightedsum

0.2870850791361813

In [64]:
model4 = PanelOLS.from_formula('pol_stab ~ con_cor + EntityEffects', data = dta_index).fit(cov_type='clustered', cluster_entity=True, group_debias=True)
print(model4)

                          PanelOLS Estimation Summary                           
Dep. Variable:               pol_stab   R-squared:                        0.0626
Estimator:                   PanelOLS   R-squared (Between):              0.6305
No. Observations:                 339   R-squared (Within):               0.0626
Date:                Mon, Jan 19 2026   R-squared (Overall):              0.6193
Time:                        22:10:10   Log-likelihood                    217.26
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      20.304
Entities:                          34   P-value                           0.0000
Avg Obs:                       9.9706   Distribution:                   F(1,304)
Min Obs:                       9.0000                                           
Max Obs:                      10.0000   F-statistic (robust):             2.6942
                            

Теперь рассмотрим процедуру взвешивания применительно к множественной регрессионной модели

Давайте адаптируем алгоритм взвешивания к случаю множественной регрессионной модели. В целом алгоритм остается тем же, но теперь нам предварительно нужно очистить вариацию зависимой переменной и интересующего нас предиктора от контрольных переменных. Для того, чтобы это сделать, нам сначала нужно оценить регрессию зависимой переменной на все контроли и сохранить остатки (то есть, информация, несвязанная с контролями). И таким же образом очищаем вариацию предиктора.

Как и в предыдущей модели, сфокусируемся на оценке коэффициента при предикторе con_cor

In [65]:
dta['y_resid'] = statf.ols(formula = 'pol_stab ~ govt_consump_WDI + herfgov_DPI + C(country)', data = dta).fit().resid
dta['x_resid'] = statf.ols(formula = 'con_cor ~ govt_consump_WDI + herfgov_DPI + C(country)', data = dta).fit().resid

dta.head()

,year,country,pol_stab,con_cor,herfgov_DPI,govt_consump_WDI,countrygroup,y_resid,x_resid
0,2006,Australia,0.935188,1.960568,0.75987,17.288634,1.0,-0.001545,0.019405
1,2007,Australia,0.928790,2.010918,0.75987,17.175808,1.0,-0.009919,0.069771
2,2008,Australia,0.955645,2.042482,1.00000,17.096951,1.0,-0.022700,0.039000
3,2009,Australia,0.855689,2.051661,1.00000,17.496081,1.0,-0.115663,0.048122
4,2010,Australia,0.888860,2.031455,1.00000,17.959095,1.0,-0.074381,0.027850


Теперь упорядочим значения вариации для x_resid, для того, чтобы понять, какие страны оказались наиболее "весомыми" для получения оценки коэффициента при con_cor, а какие - наоборот, наименее значимыми.

In [66]:
cleanvarbygroups = dta.groupby('country', sort = False).var().x_resid*(n-1)

cleanvarbygroups.sort_values(ascending = False)

,0
country,
Slovakia,0.453382
Austria,0.391332
Spain,0.383451
Hungary,0.277066
Greece,0.253874
Iceland,0.246164
Japan,0.217158
Italy,0.157475
Norway,0.148510


Посчитаем теперь взвешенную сумму оценок коэффициентов, полученных в результате оценивания моделей y_resid на x_resid по подгруппам. Убедимся, что в итоге это та же оценка коэффициента при con_cor, что мы получали и ранее в FE-модели.

In [67]:
coefconcor_multiplereg = []

for i in dta.country.unique():
  subsample = dta[dta.country == i]
  coefconcor_multiplereg.append(statf.ols(formula = 'y_resid ~ x_resid', data = subsample).fit().params.iloc[1])

In [68]:
weightedsum1 = sum(coefconcor_multiplereg*cleanvarbygroups/sum(cleanvarbygroups))
weightedsum1

0.2701970312771449

Проверим, хорошо ли предсказывает модель значения политической стабильности. При этом так как мы работаем с панельным массивом, нас будут интересовать предсказание с разбиением на подгруппы - страны. Оценим корреляцию зависимой переменной (наблюдаемого отклика) и предсказанного значения отклика по подгруппам. Если корреляция низкая (по модулю), то это говорит о том, что мы плохо объясняем моделью то, что происходит в данной группе (стране). Переоценим модель на подвыборке тех стран, для которых корреляция по модулю более либо равна 0.3 и посмотрим, устойчивы ли наши результаты.  

In [69]:
dta['ypredicted'] = model1_clustered_se.predict()

groups_data = pd.DataFrame(dta.groupby('country')['ypredicted'].corr(dta['pol_stab']))
groups_data['group'] = groups_data['ypredicted'].apply(lambda x: 1 if abs(x) >= 0.3 else 0)
data_merged = pd.merge(dta, groups_data, on = 'country')

data_merged = data_merged.set_index(['country', 'year'])
model_high = PanelOLS.from_formula('pol_stab ~ con_cor + govt_consump_WDI + herfgov_DPI + EntityEffects', data = data_merged[data_merged['group']==1]).fit(cov_type="clustered", cluster_entity=True, group_debias = True)
print(model_high)

                          PanelOLS Estimation Summary                           
Dep. Variable:               pol_stab   R-squared:                        0.1276
Estimator:                   PanelOLS   R-squared (Between):              0.3890
No. Observations:                 229   R-squared (Within):               0.1276
Date:                Mon, Jan 19 2026   R-squared (Overall):              0.3823
Time:                        22:10:11   Log-likelihood                    131.05
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      9.8930
Entities:                          23   P-value                           0.0000
Avg Obs:                       9.9565   Distribution:                   F(3,203)
Min Obs:                       9.0000                                           
Max Obs:                      10.0000   F-statistic (robust):             1.5713
                            